In [1]:
import google.generativeai as genai
import google.generativeai as genai
import ast
import os
import random
from nest_asyncio import apply
from templates import templates
import json
from image_agent import get_images
import asyncio
apply()

# Load environment variables from a .env file
from dotenv import load_dotenv

load_dotenv()

from wikipedia_tool import wikipedia_tool

# Parse API keys stored in an environment variable and convert them into a Python list
GEMINI_API_KEYS = os.environ.get("GEMINI_API_KEYS")
KEY_LIST = ast.literal_eval(GEMINI_API_KEYS)

# Shuffle the API keys list to ensure usage of different keys over time
random.shuffle(KEY_LIST)

# Initialize a global index to track the current API key being used
current_api_key_index = 0

# List to store historical messages for reference or logging
messages = []


def cycle_api_key():
    """Retrieve the next API key from the list, cycling back to the start if necessary."""
    global current_api_key_index
    if current_api_key_index >= len(KEY_LIST) - 1:
        current_api_key_index = 0
    else:
        current_api_key_index += 1
    return KEY_LIST[current_api_key_index]


def generate_new_model():
    """Generate and configure a new AI model instance with a cycled API key."""
    global current_api_key_index
    global messages
    api_key = cycle_api_key()  # Cycle to the next API key

    # Configure the generative AI model with the new API key
    genai.configure(api_key=api_key)

    # Initialize the model with specific configurations
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            temperature=0,  # Set deterministic behavior for the model
            max_output_tokens=8000
        ),
    )
    return model

/Users/artemis/miniconda3/envs/teachme/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
topic = "stacks in programing"

sources = wikipedia_tool.run(topic)
print(sources)

Page: Stack (abstract data type)
Summary: In computer science, a stack is an abstract data type that serves as a collection of elements with two main operations:

Push, which adds an element to the collection, and
Pop, which removes the most recently added element.Additionally, a peek operation can, without modifying the stack, return the value of the last element added. The name stack is an analogy to a set of physical items stacked one atop another, such as a stack of plates. 
The order in which an element added to or removed from a stack is described as last in, first out, referred to by the acronym LIFO. As with a stack of physical objects, this structure makes it easy to take an item off the top of the stack, but accessing a datum deeper in the stack may require removing multiple other items first.Considered a linear data structure, or more abstractly a sequential collection, a stack has one end which is the only position at which the push and pop operations may occur, the top of 

In [3]:
def sources_to_lecture(original_prompt, sources):
    """Converts a list of sources into a single lecture template.

    Args:
        sources (list): A list of sources, each containing a 'content' key with the source content.

    Returns:
        str: A single lecture template combining all the source content.
    """
    model = generate_new_model()

    prompt = """
Available Templates:
[
  {
    "template_id": 1,
    "num_images": 1,
    "num_texts": 3,
    "description": "Two column layout with text on the left and a single image on the right. Useful for explaining a concept with a visual aid."
  },
  {
    "template_id": 2, 
    "num_images": 1,
    "num_texts": 3,
    "description": "Two column layout with an image on the left and text on the right. Useful for leading with a visual and providing explanation."
  },
  {
    "template_id": 3,
    "num_images": 1,
    "num_texts": 3, 
    "description": "Centered layout with a title, text, and image stacked vertically. Good for focusing on a single key point."
  },
  {
    "template_id": 4,
    "num_images": 0,
    "num_texts": 3,
    "description": "A list layout with a title and bulleted list items. Useful for listing out multiple points or steps in a process."
  },
  {
    "template_id": 5,
    "num_images": 0,
    "num_texts": 3,
    "description": "A section layout with a large title and body text. Good for longer form textual content."
  },
  {
    "template_id": 6,
    "num_images": 0,
    "num_texts": 3,
    "description": "A statement layout for displaying an impactful quote or statistic with a title."
  },
  {
    "template_id": 7,
    "num_images": 0,
    "num_texts": 3,
    "description": "A big fact layout for displaying a key fact or figure with a title for emphasis."
  },
  {
    "template_id": 8,
    "num_images": 0,
    "num_texts": 2,
    "description": "A quote slide with an attribution. Displays a quote with the source."
  },
  {
    "template_id": 9,
    "num_images": 1,
    "num_texts": 0,
    "description": "A horizontal image layout with a title and description. Displays an image that bleeds horizontally off both sides."
  },
  {
    "template_id": 10,
    "num_images": 1,
    "num_texts": 3,
    "description": "A vertical image with a title and list. Displays an image with a title and bulleted list next to it."
  },
  {
    "template_id": 11,
    "num_images": 3,
    "num_texts": 0,
    "description": "A three up image layout with one primary image and two secondary images, one above and one below."
  },
  {
    "template_id": 12,
    "num_images": 1,
    "num_texts": 0,
    "description": "A full bleed image layout that displays a single image that fills the entire slide."
  }
]    
    
You are an advanced assistant that is in charge of aggregating multiple sources of information into a lecture based on a specific prompt.
Output 5 different slides on the topic given above using the sources provided as well as the given templates.
Make sure the slides flow logically and are easy to understand. Use the correct templates for the content you are presenting.

Return a json parsable string of lectures that are generated from the sources provided.
Make sure the response is in the following format:
{
    "title": title of the lecture,
    "description" : description of the lecture,
    "slides": [
        {
        "template_id": 1
        "texts" : [
            "a stack is a data structure",
            "stacks are used in DFS"
        ]
        "speaker_notes" : things that are not on the slide but are important to mention
        }   
    ]
}
"""
    lecture = model.generate_content(sources + "\n\n" + "Original Topic: " + original_prompt + "\n\n" + prompt)
    return lecture.text

result = sources_to_lecture(topic, sources)
if "```json" in result:
    # Get the JSON content from the result
    result = result.split("```json")[1]
    result = result.split("```")[0]
    
result = json.loads(result)
print(result)
    

{'title': 'Stacks in Programming: A Deep Dive', 'description': 'This lecture explores the concept of stacks, their implementation in programming, and their connection to Reverse Polish Notation.', 'slides': [{'template_id': 5, 'texts': ['Stacks: A Fundamental Data Structure', 'A stack is a LIFO (Last-In-First-Out) data structure, analogous to a stack of plates. Elements are added (pushed) and removed (popped) from the top. This structure is crucial for various algorithms and programming paradigms.'], 'speaker_notes': 'Emphasize the LIFO principle and its real-world analogy. Briefly mention applications like function call stacks and undo/redo functionality.'}, {'template_id': 1, 'texts': ['Stack Operations', '**Push:** Adds an element to the top of the stack.\n**Pop:** Removes and returns the top element.\n**Peek:** Returns the top element without removing it.'], 'image': 'image_of_stack_operations.png', 'speaker_notes': 'Visually demonstrate each operation using the image. Explain how 

In [4]:
tasks = []
new_slides = []

for slide in result['slides']:
    num_images = templates[slide['template_id']]['num_images']
    
    if num_images == 0:
        new_slides.append({
            **slide,
            "images": ""
        })
    else:
        topic = slide['speaker_notes']
        # Schedule the get_images task for concurrent execution
        task = get_images(topic, num_images)
        tasks.append(task)

# Run all tasks concurrently and collect results
images_results = await asyncio.gather(*tasks)

# Iterate over the slides that require images
image_index = 0
for slide in result['slides']:
    num_images = templates[slide['template_id']]['num_images']
    if num_images != 0:
        new_slides.append({
            **slide,
            "images": images_results[image_index]
        })
        image_index += 1
print(new_slides)

[{'template_id': 5, 'texts': ['Stacks: A Fundamental Data Structure', 'A stack is a LIFO (Last-In-First-Out) data structure, analogous to a stack of plates. Elements are added (pushed) and removed (popped) from the top. This structure is crucial for various algorithms and programming paradigms.'], 'speaker_notes': 'Emphasize the LIFO principle and its real-world analogy. Briefly mention applications like function call stacks and undo/redo functionality.', 'images': ''}, {'template_id': 4, 'texts': ['Stack Applications', '1. **Function Calls:** Stacks store return addresses and local variables during function execution.\n2. **Undo/Redo:** Stacks track user actions for easy reversal.\n3. **Expression Evaluation:** Stacks help evaluate mathematical expressions, especially in Reverse Polish Notation.\n4. **Depth-First Search (DFS):** Stacks are essential for exploring tree and graph structures.'], 'speaker_notes': 'Elaborate on each application with specific examples. Highlight the importa

In [5]:
new_slides

[{'template_id': 5,
  'texts': ['Stacks: A Fundamental Data Structure',
   'A stack is a LIFO (Last-In-First-Out) data structure, analogous to a stack of plates. Elements are added (pushed) and removed (popped) from the top. This structure is crucial for various algorithms and programming paradigms.'],
  'speaker_notes': 'Emphasize the LIFO principle and its real-world analogy. Briefly mention applications like function call stacks and undo/redo functionality.',
  'images': ''},
 {'template_id': 4,
  'texts': ['Stack Applications',
   '1. **Function Calls:** Stacks store return addresses and local variables during function execution.\n2. **Undo/Redo:** Stacks track user actions for easy reversal.\n3. **Expression Evaluation:** Stacks help evaluate mathematical expressions, especially in Reverse Polish Notation.\n4. **Depth-First Search (DFS):** Stacks are essential for exploring tree and graph structures.'],
  'speaker_notes': 'Elaborate on each application with specific examples. High